In [1]:
from Topic_Models.classical_topic_model import Topic_Model
import pandas as pd
from classifier import Active_Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from Topic_Models.neural_model import Neural_Model
import pickle
from multiprocessing import Process, Manager
import copy
from scipy.sparse import csr_matrix
import numpy as np
from scipy.sparse import csr_matrix, hstack

/nfshomes/zli12321/micromamba/envs/user_study/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
The processed document path and the raw documents path
'''

# doc_dir = './Topic_Models/Data/congressional_bill_train.json'
# processed_doc_dir = './Topic_Models/Data/congressional_bill_train_processed.pkl'
# doc_dir = './Topic_Models/Data/newsgroups/newsgroup_train.json'
# processed_doc_dir = './Topic_Models/Data/newsgroups/newsgroup_train_processed.pkl'

doc_dir = './Data/community_resilience/community_resilience_clean.json'
processed_doc_dir = './Data/community_resilience/community_resilience_processed.pkl'

'''
Mapping the mode numbers to which model we use
LA: active learning baseline
'''
model_types_map = {0: 'LA' , 1: 'LDA', 2: 'SLDA', 3: 'ETM', 4: 'CTM', 7: 'LLDA', 8: 'PLDA', 9: 'Bertopic', 10: 'SCTM'}

'''
num_iter: number of iterations to run when updating sLDA model
load_data: If use load_data, a trained topic model would be loaded.
'''
num_iter = 2000
load_data = True
save_model = False

'''
Enter the number of topics for the model you just trained
'''
num_topics = 35
inference_alg = 'logreg'
# test_dataset_name = './Topic_Models/Data/congressional_bill_test.json'
# test_processed_doc_dir = './Topic_Models/Data/congressional_bill_test_processed.pkl'
test_dataset_name = './Data/community_resilience/community_resilience_clean.json'
test_processed_doc_dir = './Data/community_resilience/community_resilience_processed.pkl'

'''
Keep those and don't change
'''
USE_TEST_DATA = True
USE_PROCESSED_TEXT = False
CONCATENATE_KEYWORDS = True
table = pd.read_json(doc_dir)
training_length = len(table)
REGRESSOR_PREDICT = True

with open(processed_doc_dir, 'rb') as inp:
    loaded_data = pickle.load(inp)
    processed_data = loaded_data['datawords_nonstop']
    list_of_preprocessed_documents = [' '.join(ele) for ele in processed_data]

raw_texts = table.text.values.tolist()

mode = 1

In [4]:
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1, 1))
vectorizer_idf = vectorizer.fit_transform(raw_texts)
vectorizer_idf

<796x6675 sparse matrix of type '<class 'numpy.float64'>'
	with 51337 stored elements in Compressed Sparse Row format>

In [5]:
model = Topic_Model(num_topics, 0, load_data_path = processed_doc_dir, model_path='./Topic_Models/trained_models/LDA_35.pkl', load_model=True)
# model = Neural_Model('./Model/ETM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)

In [6]:
'''
find_doc_for_TA: Given a dictionary, where the keys are topic numbers
the values are list of tuples of (document_id, probability), first find
the topic that has the maximum median probability, then finds the document
within that topic that has the maximum probaiblity
'''
from utils.tools import active_selection
import numpy as np

In [7]:
'''
The document topic distribution for the topic model
'''
document_probas = model.document_probas

In [8]:
'''
The result is a N X K matrix. N is the number of
documents, K is the number of topics
'''
np.shape(model.doc_topic_probas)

(796, 35)

In [9]:
def count_docs(dictionary):
    result = 0
    for k, v in dictionary.items():
        result += len(v)

    return result

def update_median_prob(topic_num, idx_in_topic, doc_probs):
    try:
        doc_probs[topic_num].pop(idx_in_topic)
    except:
          doc_probs.pop(topic_num)

def print_topic_lens(dictionary):
    for k, v in dictionary.items():
        print('topic {} num docs {}'.format(k, len(v)))

In [10]:
%%capture
print('total num docs', count_docs(document_probas))

recommended_docs = []
recommended_topics = []
# Since the length of the document probas change, np.ones length should also change
for i in range(100):
    chosen_idx, chosen_topic, chosen_idx_in_topic = active_selection(document_probas, np.ones(training_length))
    update_median_prob(chosen_topic, chosen_idx_in_topic, document_probas)
    # print_topic_lens(document_probas)
    print('doc id is ', chosen_idx)
    # chosen_topic
    recommended_topics.append(chosen_topic)
    recommended_docs.append(chosen_idx)

In [11]:
'''
Run a hundred iterations to pick the document that has the maximum median probability in the topic.
Check the distribution of topics being picked. This is a good way to check topic diversity first
'''

from collections import Counter
Counter(recommended_topics)

Counter({1: 6,
         31: 2,
         2: 11,
         4: 10,
         30: 6,
         17: 6,
         28: 3,
         6: 8,
         13: 2,
         33: 13,
         19: 4,
         26: 4,
         21: 3,
         32: 6,
         34: 1,
         11: 3,
         10: 8,
         22: 3,
         9: 1})

In [12]:
import numpy as np
from tqdm import tqdm
def save_results(lst, file_name):
    a_obj = np.array(lst, dtype=object)
    np.save(file_name, a_obj)

In [13]:
'''
Active learning baseline to test the metrics of clf without topic modeling
'''
def calculate_activelearning_diversity(process=False, unigram=False):
    accs, purity, ri, nmi = [], [], [], []
    test_accs, test_purity, test_ri, test_nmi = [], [], [], []
    test_df = pd.read_json(test_dataset_name)
    # print('test length is', len(test_df))

    '''
    Read the documents and the labels
    '''
    df = pd.read_json(doc_dir)
    labels = df.sub_labels.values.tolist()

    raw_texts = df.text.values.tolist()[0:training_length]
    test_texts = test_df.text.values.tolist()
    # raw_texts.extend(test_texts)


    '''
    Tried to test the performance of unigram and bigrams, the performance of bigram vectorizer is better
    '''
    if unigram == True:
        vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1, 1))
    else:
        vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1, 2))
    '''
    If process is True, load the processed data and use processed data as inputs for the clf
    else, just use the raw texts to as input for the clf. Results indicate that using raw texts
    has higher performance than using processed data
    '''
    if not process:
        vectorizer_idf = vectorizer.fit_transform(raw_texts)
        test_vecorizer = vectorizer.transform(test_texts)
    else:
        with open(processed_doc_dir, 'rb') as inp:
            loaded_train_data = pickle.load(inp)
            processed_train_data = loaded_train_data['datawords_nonstop']


        text_data = [' '.join(doc) for doc in processed_train_data]
        # from sklearn.feature_extraction.text import CountVectorizer
        # vectorizer = CountVectorizer(binary=True)
        vectorizer_idf = vectorizer.fit_transform(text_data)

    # vectorizer_idf = bow_context_idf
    '''
    Initialize the active learning classifier
    '''
    alto = Active_Learning(raw_texts, None,  None, df, inference_alg, vectorizer_idf, training_length, 0, test_df, test_vecorizer)

    recommended_docs1 = []
    recommended_topics1 = []
    scores = []

    '''
    Label 300 hundred documents and save the three metrics
    '''
    for i in tqdm(range(510)):
        '''
        Get the recommended document id and the score (entropy) for the current document
        '''
        recommend_id, score = alto.recommend_document(True)

        alto.label(recommend_id, labels[recommend_id])
        recommended_docs1.append(recommend_id)
        recommended_topics1.append(alto.last_recommended_topic)
        scores.append(score)
        # print(raw_texts[recommend_id])

        '''
        clf only works for at least 2 distinct classes
        '''
        if len(alto.classes) >= 3:
            train_acc, b, p, r, n, e, f, g = alto.eval_classifier()
            accs.append(train_acc)
            purity.append(p)
            ri.append(r)
            nmi.append(n)
            test_accs.append(b)
            test_purity.append(e)
            test_ri.append(f)
            test_nmi.append(g)

    return accs, purity, ri, nmi, test_accs, test_purity, test_ri , test_nmi

In [20]:
'''
Evaluate the performance of activalearning clf with topic model features added to the clf
'''
def calculate_topic_diversity(module, concat, topic_features_only, user_num, Model=None):
    model_topics = []
    accs, purity, ri, nmi = [], [], [], []
    test_accs, test_purity, test_ri, test_nmi = [], [], [], []
    test_df = pd.read_json(test_dataset_name)
    # print('test length is', len(test_df))

    if Model is None:
        if module == 3:
            model = Neural_Model('./Topic_Models/trained_models/ETM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        elif module == 4:
            model = Neural_Model('./Topic_Models/trained_models/CTM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        elif module == 9:
            model = Neural_Model('./Topic_Models/trained_models/Bertopic_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        elif module == 10:
            model = Neural_Model('./Topic_Models/trained_models/SCTM_{}.pkl'.format(num_topics), processed_doc_dir, doc_dir)
        else:
            model = Topic_Model(num_topics, 0, load_data_path = processed_doc_dir, model_path='./Topic_Models/trained_models/LDA_35.pkl', load_model=True, model_type= 'LDA')
    else:
        '''
        If no module is selected, then use the Model passed in in the parameter
        '''
        model = Model

    '''
    Read texts, construct a text encoding for the clf
    '''
    df = pd.read_json(doc_dir)

    df1 = pd.read_csv('./01-user_data/user_data_per_doc/{}.csv'.format(user_num))
    labels = df1.label.values.tolist()
    made_labels = df.label.values.tolist()
    raw_texts = df.text.values.tolist()[0:training_length]
    document_probas, doc_topic_probas = model.group_docs_to_topics()
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1,1))

    '''
    If concatenation is true, concatenate the topic distribution with the text features
    '''
    if concat == True:
        if topic_features_only == True:
            vectorizer_idf = csr_matrix(model.doc_topic_probas).astype(np.float64)
        else:
            vectorizer_idf = vectorizer.fit_transform(raw_texts)

            '''
            concatenate the features from topic model with the clf
            '''
            concatenated_features = model.concatenate_features(model.doc_topic_probas, vectorizer_idf)

            vectorizer_idf = concatenated_features
    else:
        model_concate_texts = raw_texts
        vectorizer_idf = vectorizer.fit_transform(model_concate_texts)



    alto = Active_Learning(copy.deepcopy(document_probas), vectorizer_idf, doc_topic_probas)

    print('start synthetic labeling')
    recommended_docs1 = []
    recommended_topics1 = []
    scores = []

    '''
    Label 300 documents
    '''

    recommended = df1.doc_id.values.tolist()
    iter_length = len(labels)
    made_labels_track = []
    
    for i in tqdm(range(iter_length)):
        # recommend_id, score = alto.recommend_document(True)
        recommend_id = recommended[i]


        alto.label(recommend_id, labels[i])
        made_labels_track.append(made_labels[i])
        recommended_docs1.append(recommend_id)
        recommended_topics1.append(alto.last_recommended_topic)


        if len(alto.classes) >= 3:
            p, r, n= alto.eval_classifier(made_labels)
            purity.append(p)
            ri.append(r)
            nmi.append(n)
            print('Purity {}; ri: {}; nmi: {}'.format(p, r, n))

    return recommended_topics1, 0, purity, ri, nmi, 0, 0, 0, 0

In [15]:
'''
Given a matrix, make its rows consistent length
'''
def make_consistent_rows(matrix):
    # Find the length of the shortest row
    min_length = min(len(row) for row in matrix)

    # Trim each row to the length of the shortest row
    a_uniform = [row[-min_length:] for row in matrix]
    a_np = np.array(a_uniform)
    return a_np

In [16]:
def run_experiment_and_save(module, save_path, topic_features_only=True):
  '''
  module: model_types_map = {0: 'LA' , 1: 'LDA', 2: 'SLDA', 3: 'ETM', 4: 'CTM', 7: 'LLDA', 8: 'PLDA', 9: 'Bertopic'}
  Enter the module number to run the the model
  '''

  if module == 0:
    start_time = time.time()
    a0, p0, r0, n0, ta0, tp0, tr0, tn0 = calculate_activelearning_diversity(False, True)
    end_time = time.time()
    result_lst = [a0, p0, r0, n0, ta0, tp0, tr0, tn0, [[end_time-start_time]]]
    save_results(result_lst, save_path)
  else:
    start_time = time.time()
    recommended_topics1, a1, p1, r1, n1, ta1, tp1, tr1, tn1 = calculate_topic_diversity(module, True, topic_features_only)
    end_time = time.time()
    result_lst = [a1, p1, r1, n1, ta1, tp1, tr1, tn1, [[end_time-start_time]]]
    print('purity list length is ', len(p1))
    save_results(result_lst, save_path)


In [22]:
'''
Run the experiment over n runs then take the medium of each run to 
save
'''
import time
def run_experiment_over_n_runs(module, save_path, n_runs, user_num, topic_features_only=True):
     a, p, r, n, ta, tp, tr, tn = [], [], [], [], [], [], [], []

     running_time = []
     if module == 0:
          for i in range(n_runs):
               start_time = time.time()
               a0, p0, r0, n0, ta0, tp0, tr0, tn0 = calculate_activelearning_diversity(False, True)
               end_time = time.time()
               a.append(a0)
               p.append(p0)
               r.append(r0)
               n.append(n0)
               ta.append(ta0)
               tp.append(tp0)
               tr.append(tr0)
               tn.append(tn0)
               running_time.append(end_time-start_time)
     else:
          for i in range(n_runs):
               start_time = time.time()
               recommended_topics1, a1, p1, r1, n1, ta1, tp1, tr1, tn1 = calculate_topic_diversity(module, True, topic_features_only, user_num)
               end_time = time.time()

               a.append(a1)
               p.append(p1)
               r.append(r1)
               n.append(n1)
               ta.append(ta1)
               tp.append(tp1)
               tr.append(tr1)
               tn.append(tn1)
               running_time.append(end_time-start_time)

     a = make_consistent_rows(a)
     p = make_consistent_rows(p)
     r = make_consistent_rows(r)
     n = make_consistent_rows(n)
     ta = make_consistent_rows(ta)
     tp = make_consistent_rows(tp)
     tr = make_consistent_rows(tr)
     tn = make_consistent_rows(tn)

     result_lst = [a, p, r, n, ta, tp, tr, tn, [running_time]]
     save_results(result_lst, save_path)

In [29]:
# %%capture
'''
Run the baseline experiment. The saved results can be plotted in the new_mode_plot.ipynb notebook
'''
# run_experiment_over_n_runs(0, './model_testing_results/congressional_bills/8_major_40_minor/baseline_contextual_bow_15_runs.npy', 15, False)
# run_experiment_over_n_runs(0, './model_testing_results/regularized_log_testing/baseline_test_docs_regularized_5_runs.npy', 5, False)
# run_experiment_over_n_runs(0, './model_testing_results/bills/bills_sub_baseline.npy', 5, False)

'\nRun the baseline experiment. The saved results can be plotted in the new_mode_plot.ipynb notebook\n'

In [32]:
# %%capture
'''
Run the LDA experiment. The saved results can be plotted in the new_mode_plot.ipynb notebook
'''
# run_experiment_and_save(1, './model_testing_results/congressional_bills/8_major_30_minor/batch_update/LDA_update_10_one_run.npy', False)
# run_experiment_over_n_runs(1, './model_testing_results/congressional_bills/8_major_40_minor/LDA_15_runs.npy', 15, False)
run_experiment_over_n_runs(1, './model_testing_results/community_resilience/35_LDA.npy', 1, 8, False)

start synthetic labeling


100%|██████████| 4/4 [00:00<00:00, 114.13it/s]

Purity 0.2964824120603015; ri: 0.4352422489807528; nmi: 0.02289814762146124
Purity 0.3165829145728643; ri: 0.5516355361714231; nmi: 0.03461925515634456


TypeError: object of type 'int' has no len()

In [22]:
%%capture
'''
Run the sLDA experiment
'''
# run_experiment_and_save(2, './model_testing_results/congressional_bills/8_major_30_minor/25_topics_SLDA_train_docs_update_regularized_one_run.npy', False)
run_experiment_over_n_runs(2, './model_testing_results/congressional_bills/8_major_40_minor/SLDA_15_runs.npy', 15, False)
# run_experiment_over_n_runs(2, './model_testing_results/20newsgroup/regularized_log_testing/30_topics_SLDA_test_docs_regularized_5_runs.npy', 5, False)

In [18]:
# %%capture
'''
Run the CTM experiment
'''
# run_experiment_and_save(4, './model_testing_results/congressional_bills/8_major_30_minor/batch_update/CTM_update_10_one_run.npy', False)
# run_experiment_over_n_runs(4, './model_testing_results/regularized_log_testing/16_topics_CTM_test_docs_regularized_5_runs.npy', 5, False)
# run_experiment_over_n_runs(4, './model_testing_results/congressional_bills/8_major_40_minor/CTM_15_runs.npy', 15, False)
run_experiment_over_n_runs(4, './model_testing_results/bills/bills_sub_CTM.npy', 1, False)

start synthetic labeling


  0%|          | 0/510 [00:00<?, ?it/s]

picked doc is  2381
[]
picked doc is  25856
['Elementary & Secondary']
max topic index is  27
max median topic is  27
['Elementary & Secondary', 'Intergovernmental Relations']
[[ 420  185  175]
 [ 366  365   41]
 [   4    9   11]
 [1282  717  426]
 [1173 1192  290]
 [ 526 1646  117]
 [ 869  764  252]
 [ 631  439  262]
 [ 720 1253  163]
 [ 775  968  778]
 [2882  777  112]
 [ 215  219   74]
 [ 362  546  103]
 [ 440  270  248]
 [ 292  581   79]
 [ 620 1087  169]
 [ 268 1158   93]
 [ 426  412 1937]
 [ 284  492   46]
 [ 338  205   50]
 [ 540  356  158]]


  1%|          | 3/510 [00:00<00:50, 10.14it/s]

train acc 0.036009553555024804; purity 0.1979606834466287; RI 0.03715428117763622; NMI 0.07405058363720972
test acc -1; purity -1; RI -1; NMI -1
max topic index is  29
max median topic is  29
['Elementary & Secondary', 'Intergovernmental Relations', 'Drug Industry']
[[ 213   55   18  494]
 [  16   70    8  678]
 [   0    0    3   21]
 [  24  169   65 2167]
 [ 185  106   14 2350]
 [   8  637   10 1634]
 [  21   54   16 1794]
 [  31   79   21 1201]
 [  76 1420    8  632]
 [  11  104  437 1969]
 [ 630  166   21 2954]
 [   4   39    7  458]
 [   1  106    8  896]
 [  14   44   21  879]
 [   8   38    3  903]
 [ 112  331   48 1385]
 [  14  110    7 1388]
 [   5   51  174 2545]
 [   3   44    4  771]
 [   9   34    1  549]
 [   1   61   24  968]]
train acc 0.059158552268969315; purity 0.1666054259293282; RI 0.007665613415613974; NMI 0.0825816395693228
test acc -1; purity -1; RI -1; NMI -1
max topic index is  41
max median topic is  41
['Elementary & Secondary', 'Intergovernmental Relations',

...

[[  0   1   2 ...   0   0   0]
 [ 12   0   4 ...   1   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  2   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [  2   0 379 ...   0   1   0]]
train acc 0.5425929328189112; purity 0.6854981933982485; RI 0.30359462457440667; NMI 0.4669373903947531
test acc -1; purity -1; RI -1; NMI -1
max topic index is  42
max median topic is  42
['Elementary & Secondary', 'Intergovernmental Relations', 'Drug Industry', 'Tax Code', 'Natural Gas & Oil', 'Nuclear Arms', 'Political Campaigns', 'Public Lands', 'Court Administration', 'Species & Forest', 'Agencies', 'Long-term Care', 'Banking', 'Right to Privacy', 'Personnel Issues', 'Postal Service', 'Small Businesses', 'Maritime', 'Subsidies to Farmers', 'Consumer Finance', 'Bureaucracy', 'Disease Prevention', 'Employee Benefits', 'Tariff & Imports', 'Child Abuse', 'Electricity', 'Broadcast', 'Labor Unions', 'Higher', 'Foreign Operations', 'Weather Forecasting', 'Family Issues', 'Worker Safety', 'Food Inspe

 89%|████████▉ | 453/510 [34:56<05:25,  5.71s/it]

[[  0   1   2 ...   0   0   0]
 [ 12   0   4 ...   1   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  2   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [  2   0 378 ...   0   1   0]]
train acc 0.542837895768265; purity 0.6854981933982485; RI 0.30356530049553576; NMI 0.46727714034057144
test acc -1; purity -1; RI -1; NMI -1
max topic index is  13
max median topic is  13
['Elementary & Secondary', 'Intergovernmental Relations', 'Drug Industry', 'Tax Code', 'Natural Gas & Oil', 'Nuclear Arms', 'Political Campaigns', 'Public Lands', 'Court Administration', 'Species & Forest', 'Agencies', 'Long-term Care', 'Banking', 'Right to Privacy', 'Personnel Issues', 'Postal Service', 'Small Businesses', 'Maritime', 'Subsidies to Farmers', 'Consumer Finance', 'Bureaucracy', 'Disease Prevention', 'Employee Benefits', 'Tariff & Imports', 'Child Abuse', 'Electricity', 'Broadcast', 'Labor Unions', 'Higher', 'Foreign Operations', 'Weather Forecasting', 'Family Issues', 'Worker Safety', 'Food Inspe

 89%|████████▉ | 453/510 [35:39<04:29,  4.72s/it]


KeyboardInterrupt: 

In [37]:
%%capture
'''
Run the CTM experiment
'''
run_experiment_and_save(10, './model_testing_results/16_topics_SCTM_test_docs_regularized_one_run.npy', False)
# run_experiment_over_n_runs(4, './model_testing_results/CTM_500_docs_5_runs_16topics.npy', 5)

In [28]:
%%capture
'''
Run the ETM experiment
'''
run_experiment_and_save(3, './model_testing_results/16_topics_ETM.npy')

FileNotFoundError: [Errno 2] No such file or directory: './Topic_Models/Model/ETM_16.pkl'

In [17]:
%%capture
'''
Run the CTM experiment
'''
run_experiment_and_save(4, './model_testing_results/50_topics_CTM_topic_features_only.npy')

In [30]:
%%capture
'''
Run the PLDA experiment
'''
run_experiment_and_save(8, './model_testing_results/16_topics_PLDA.npy')

KeyboardInterrupt: 

In [21]:
%%capture
'''
Run the Bertopic experiment
'''
run_experiment_and_save(9, './model_testing_results/50_topics_BERTOPIC_topic_features_only.npy')

### Fetch word priors from the topic model. Possibly can be used as features for the classifier

In [ ]:
from topic_model import Topic_Model
full_len = len(pd.read_json('./Data/congressional_bill_train.json'))
Model = Topic_Model(13, 2500, 'LDA', './Data/congressional_bill_train_processed.pkl', full_len , {}, False, None)
save_path = './Model/{}_{}.pkl'.format('LDA', 13)
Model.train(save_path)

ModuleNotFoundError: ignored

In [ ]:
len(Model.model.used_vocabs)

9306

In [ ]:
Model.model.used_vocabs

['program', 'united', 'national', 'establish', 'authorize', 'code', 'section', 'title', 'public', 'state', 'health', 'service', 'require', 'housing', 'secretary', 'development', 'relate', 'energy', 'assistance', 'education', 'tax', 'use', 'increase', 'law', 'grant', 'income', 'year', 'revenue', 'improve', 'respect', 'extend', 'land', 'fund', 'individual', 'security', 'internal', 'system', 'prohibit', 'provision', 'department', 'loan', 'control', 'water', 'school', 'commission', 'amend', 'care', 'agency', 'project', 'area', 'facility', 'agricultural', 'research', 'benefit', 'provide', 'construction', 'local', 'person', 'government', 'protection', 'promote', 'transportation', 'labor', 'employee', 'administration', 'authority', 'additional', 'conservation', 'protect', 'permit', 'assist', 'employment', 'shall', 'commerce', 'payment', 'safety', 'community', 'establishment', 'new', 'insurance', 'appropriation', 'drug', 'right', 'requirement', 'child', 'order', 'information', 'interstate', 'd

In [ ]:
for word in Model.model.used_vocabs:
    prior = Model.model.get_word_prior(word)
    if len(prior) > 0:
        print(word)

In [ ]:
Model.model.tw

0

In [ ]:
model_vocabs = set()
for i in Model.data_words_nonstop:
    for j in i:
        model_vocabs.add(j)

len(model_vocabs)

9306

In [ ]:
from gensim import corpora, models

# Example corpus of documents (list of lists of tokens)
with open('./Data/congressional_bill_train_processed.pkl', 'rb') as inp:
    data = pickle.load(inp)
    corpus = data['datawords_nonstop']

# Create a dictionary from the corpus
dictionary = corpora.Dictionary(corpus)

# Create a bag-of-words representation of the corpus
bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]

# Train an LDA model on the bag-of-words corpus
lda_model = models.LdaModel(bow_corpus, num_topics=13, id2word=dictionary, passes=10)



Word Prior Probability for 'college': 0.0000


In [ ]:
def get_prior(target_word):
    # The word for which you want to get the word prior probability
    # target_word = "veteran"

    # Get the word prior probability for the target word
    word_id = dictionary.token2id[target_word]
    word_prior_prob = sum(prob for _, prob in lda_model.get_term_topics(word_id))

    print(f"Word Prior Probability for '{target_word}': {word_prior_prob:.4f}")

In [ ]:
for word in corpus[0]:
    get_prior(word)

Word Prior Probability for 'authorize': 0.0886
Word Prior Probability for 'study': 0.0000
Word Prior Probability for 'feasibility': 0.0000
Word Prior Probability for 'desirability': 0.0000
Word Prior Probability for 'establish': 0.0602
Word Prior Probability for 'national': 0.0994
Word Prior Probability for 'recreation': 0.0000
Word Prior Probability for 'area': 0.0113
Word Prior Probability for 'know': 0.0000
Word Prior Probability for 'santa': 0.0000
Word Prior Probability for 'margarita': 0.0000
Word Prior Probability for 'national': 0.0994
Word Prior Probability for 'recreation': 0.0000
Word Prior Probability for 'area': 0.0113
Word Prior Probability for 'area': 0.0113
Word Prior Probability for 'san': 0.0000
Word Prior Probability for 'diego': 0.0000
Word Prior Probability for 'county': 0.0145
Word Prior Probability for 'calif': 0.0000
Word Prior Probability for 'presently': 0.0000
Word Prior Probability for 'constitute': 0.0000
Word Prior Probability for 'camp': 0.0000
Word Prior